In [34]:
%matplotlib inline

import sys
import os
import numpy as np
import pandas as pd
import xarray as xr
from math import ceil,sin,cos, atan2, pi, sqrt
import matplotlib
from matplotlib import pyplot as plt, animation, rc,colors
from datetime import datetime, timedelta
from cartopy import crs, feature
import cmocean
import yaml

from IPython.display import Image
rc('animation', html='html5')

sys.path.append('/home/jvalenti/MOAD/analysis-jose/Source')
from OP_functions import *

In [35]:
path_d='/results2/SalishSea/nowcast-green.201905/31oct22/SalishSea_1h_20221031_20221031_carp_T.nc'
path_area='/results2/SalishSea/nowcast-green.201905/31oct22/SalishSea_1h_20221031_20221031_grid_T.nc'

e3t = xr.open_dataset(path_d,decode_times=False).e3t
Area = xr.open_dataset(path_area,decode_times=False).area

In [54]:
#np.multiply(e3t,Area)

In [36]:
# Define paths
local = 0 #Set to 0 when working on server
paths = path(local)
config='/home/jvalenti/MOAD/analysis-jose/OParcels/neutral_iona.yaml'

In [37]:
param = load_config1(config)
start = datetime(param['startdate']['year'], param['startdate']['month'], param['startdate']['day']) #Start date
Tmax = param['param']['length'] # Set Time length [days] 
duration = timedelta(days=Tmax)
dt = param['param']['dt'] #toggle between - or + to pick backwards or forwards 
N = param['param']['N'] # number of deploying locations
n = param['param']['n'] # 1000   # number of particles per location
dmin = param['param']['dmin'] #minimum depth
dd = param['param']['dd'] #max depth difference from dmin
name = param['file']['name'] #name output file

daterange = [start+timedelta(days=i) for i in range(Tmax)]
fn =  name + '_'.join(d.strftime('%Y%m%d')+'_1n' for d in [start, start+duration]) + '.nc'
outfile = os.path.join(paths['out'], fn)

In [58]:
def paths_vol (start,duration,dt):
    Arealist = []
    e3tlist = []
    for day in range(duration.days):
            path_NEMO = make_prefix(start + timedelta(days=day), paths['NEMO'])
            Arealist.append(path_NEMO + '_grid_T.nc')
            e3tlist.append(path_NEMO + '_carp_T.nc')
    return Arealist, e3tlist
Arealist, e3tlist = paths_vol (start,duration,dt)


In [40]:
def paths_vol (start,duration):
    Arealist = []
    e3tlist = []
    for day in range(duration.days):
            path_NEMO = make_prefix(start + timedelta(days=day), paths['NEMO'])
            Arealist.append(path_NEMO + '_grid_T.nc')
            e3tlist.append(path_NEMO + '_carp_T.nc')
    return Arealist, e3tlist
Arealist, e3tlist = paths_vol (start,duration)


In [55]:
# def Volume_cells(start,dt):
#     opsdir, namfmt, jlims, ilims = getWindVars(start)
#     flist=et.index_model_files(start,start+DeltaT,opsdir,namfmt,
#                             flen=1,ftype='None',tres=24);
#     fs=[xr.open_dataset(ii,chunks='auto') for ii in flist['paths']]
#     fu=xr.concat([ii['u_wind'].isel(x=slice(ilims[0],ilims[1]),y=slice(jlims[0],jlims[1])) for ii in fs],"time_counter")
#     fv=xr.concat([ii['v_wind'].isel(x=slice(ilims[0],ilims[1]),y=slice(jlims[0],jlims[1])) for ii in fs],"time_counter")
#     fud=xr.concat([ii['u_wind'].isel(x=72,y=62) for ii in fs],"time_counter")
#     fvd=xr.concat([ii['v_wind'].isel(x=72,y=62) for ii in fs],"time_counter")
#     Week1=(fu**2+fv**2)**(3/2)
#     wind=(fu**2+fv**2)**(1/2)
#     umean = np.nanmean(np.mean(fu,0))
#     vmean = np.nanmean(np.mean(fv,0))
#     Windir = np.arctan2(vmean,umean)
#     Week1mean=np.mean(Week1,0)
#     return np.nanmean(Windir), np.nanmean(Week1mean),np.nanmax(wind)
# Qwind = [] 
# Windir = [] 
# Mwind = []
# for i in range(1,104):
#     Wdr,Qww,Wmm = Qubicwind_tmean(start, timedelta(days= 7))
#     Mwind.append(Wmm)
#     Qwind.append(Qww)
#     Windir.append(Wdr)
#     start = start+timedelta(days= 7)